In [12]:
%matplotlib widget
#%matplotlib inline

In [13]:
import numpy as np
import pandas as pd
import aseg_gdf2
from shapely.geometry import Point, LineString, MultiPoint, Polygon
from shapely.wkt import loads
from scipy.io import loadmat
from scipy import spatial
import matplotlib.pyplot as plt
import rasterio
import netCDF4
import h5py
import gc
import os
import glob
from scipy.spatial.ckdtree import cKDTree
from hydrogeol_utils import SNMR_utils, spatial_functions, AEM_utils
from hydrogeol_utils import plotting_utils as plot_utils

In [14]:
def nearest_neighbours(points, coords, points_required = 1, max_distance = 250.):
    """

    :param points: array of points to find the nearest neighbour for
    :param coords: coordinates of points
    :param points_required: number of points to return
    :param max_distance: maximum search radius
    :return:
    """
    # Initialise tree instance
    kdtree = cKDTree(data=coords)

    # iterate throught the points and find the nearest neighbour
    distances, indices = kdtree.query(points, k=points_required,
                                      distance_upper_bound=max_distance)
    print(distances)
    # Mask out infitnite distances in indices to avoid confusion
    mask = ~np.isfinite(distances)

    distances[mask] = np.nan

    return distances, indices

# Find the nearest neighbours within the maximum distance

def xy_2_var(grid_dict, xy, var):
    """
    Function for finding a variable for gridded AEM sections
    given an input easting and northing
    @ param: grid_dict :dictionary for gridded line data
    @ param: xy: numpy array with easting and northing
    @ param: var: string with variable name
    returns
    float: distance along line
    """
    utm_coords = np.column_stack((grid_dict['easting'],
                                  grid_dict['northing']))

    d, i = spatial_functions.nearest_neighbours(xy,
                                                utm_coords,
                                                points_required=1,
                                                max_distance=100.)
    if np.isnan(d[0]):
        return None

    else:
        near_ind = i[0]
    


        return grid_dict[var][near_ind]


In [42]:
# bring in the inversion ready data

infile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\gadi\combined\outputs\lines\rjmcmc"

gdf = aseg_gdf2.read(infile).df()

In [43]:
gdf['fiducial ']

0      2415766.0
1      2415771.0
2      2415933.0
3      2416683.5
4      2416688.5
         ...    
365    3119267.0
366    3119272.0
367    3119277.0
368    3034156.5
369    3034161.5
Name: fiducial , Length: 370, dtype: float64

In [44]:
# Now add the path to the netcdf file based on the naming convention

root = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\gadi\combined\outputs\pmaps"


gdf['pmap_path']= ''

for index, row in gdf.iterrows():
    fn= ['seq', str(row['uniqueid ']).zfill(8), str(row['line ']), "{:.6f}".format(row['fiducial ']), 'nc' ]
    gdf.at[index, 'pmap_path'] = os.path.join(root, '.'.join(fn))

In [18]:
# This column is for interpreting the Oolloo-Jinduckin interface

gdf['dyke_B_upper'] = np.nan
gdf['dyke_B_lower'] = np.nan

In [45]:
lines = gdf['line '].unique()

In [46]:
infile = r"C:\Users\PCUser\Desktop\AEM\LCI\HowardE_WB_MGA52.nc"

lci_dat = netCDF4.Dataset(infile)

lci_coords = np.column_stack((lci_dat['easting'][:],
                          lci_dat['northing'][:]))

# Initialise tree instance for nearest neighbour searches
kdtree = cKDTree(data=lci_coords)


In [47]:
# Since we will be wanting to plot the sections lets first grid them

# Create an instance of plots for gridding the data

plots = plot_utils.ConductivitySectionPlot(lci_dat)


# Define some key variables which we want to inteprolate

cond_vars = ['conductivity', 'data_residual', 'depth_of_investigation']

plots.conductivity_variables = cond_vars


# Define the resolution of th sections
xres, yres = 16., 4.


In [22]:

# Now grid the lines and save in memory

hdf5_dir = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\lci\hdf5"

if not os.path.exists(hdf5_dir):
    os.mkdir(hdf5_dir)

gridded_vars ={}

# Saved these out to save time

#gridded_vars = plots.grid_variables(xres = xres, yres =yres, lines=lines,
#                                    resampling_method = 'linear', save_hdf5 = True,
#                                   return_dict = True, hdf5_dir = hdf5_dir)

for line in lines:
    
    infile = os.path.join(hdf5_dir, str(line) + '.hdf5')
    f = h5py.File(infile, 'r')
    gridded_vars[line] = plot_utils.extract_hdf5_data(f, cond_vars)
    f = None
    gc.collect()

In [48]:
gdf

,uniqueid,survey,date,flight,line,fiducial,easting,northing,elevation,nchains,...,changepoint [141],changepoint [142],changepoint [143],changepoint [144],changepoint [145],changepoint [146],changepoint [147],changepoint [148],changepoint [149],pmap_path
0,1,1303,20170728,20170729,107501,2415766.0,732671.5,8626858.0,18.4,8,...,39,50,35,40,41,36,33,19,26,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...
1,11,1303,20170728,20170729,107501,2415771.0,732764.3,8626960.0,19.1,8,...,6,7,9,2,6,7,8,4,2,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...
2,21,1303,20170728,20170729,107601,2415933.0,732774.1,8626822.0,19.6,8,...,1,2,2,0,1,2,3,4,1,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...
3,31,1303,20170728,20170729,107701,2416683.5,732799.9,8626712.0,18.4,8,...,1,1,0,0,0,0,0,1,1,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...
4,41,1303,20170728,20170729,107701,2416688.5,732887.3,8626809.0,20.6,8,...,0,1,2,0,1,0,0,1,0,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,1805,1303,20170806,20170806,200701,3119267.0,734306.9,8624239.0,26.1,8,...,1,2,0,2,3,2,0,2,1,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...
366,1815,1303,20170806,20170806,200701,3119272.0,734196.9,8624334.0,26.9,8,...,2,3,0,1,0,2,0,0,1,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...
367,1825,1303,20170806,20170806,200701,3119277.0,734084.5,8624417.0,27.2,8,...,0,1,3,0,2,0,2,3,0,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...
368,1835,1303,20170805,20170805,200801,3034156.5,733590.0,8625245.0,26.4,8,...,0,1,0,0,0,1,1,2,0,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...


In [49]:
# Add the distance along the gridded lines to the dataframe

gdf['dist_along_line'] = np.nan

gdf['point_ind_lci'] = -9999


for index, row in gdf.iterrows():

    # get the line and coordinates
    line = row['line ']
    
    xy = np.array([row[['easting ','northing ']].values])
    
    gdf.loc[index,'dist_along_line'] = xy_2_var(gridded_vars[line],
                                               xy,'grid_distances')
    
    gdf.loc[index,'point_ind_lci'] = kdtree.query(row[['easting ', 'northing ']].values)[1]
    


In [50]:
gridded_vars.keys()

dict_keys([107501, 107601, 107701, 107801, 107901, 108001, 108101, 108201, 108301, 108401, 108501, 108601, 108701, 108801, 108901, 109001, 109101, 109201, 109301, 109401, 109501, 109601, 109701, 109801, 109901, 110001, 110101, 110201, 110301, 110401, 110501, 110502, 110601, 110701, 110801, 110901, 111002, 111101, 111201, 111301, 111401, 111501, 111601, 111701, 111801, 111901, 112001, 112101, 112201, 112301, 112401, 112501, 112601, 112701, 112801, 112901, 113001, 113101, 113201, 113301, 113401, 113501, 113601, 113701, 113801, 113901, 114001, 114101, 114201, 114301, 114401, 114501, 114601, 114701, 114801, 114901, 115001, 115101, 115201, 115301, 115451, 115551, 115651, 115751, 115851, 115951, 116051, 116151, 116251, 116351, 116451, 116551, 116651, 116701, 116801, 116901, 117001, 117101, 117201, 117301, 117401, 117501, 200301, 200401, 200501, 200601, 200701, 200801])

In [51]:
# Now we bring in the AEM grids

inRaster = r"C:\Users\PCUser\Desktop\2017_HowardsEast_SkyTEM\03_LCI\03_Depth_Slices\Grids_doi_Masked\*.ers"

cond = {}

for file in glob.glob(inRaster):
    layer = int(file.split('Con')[1].split('_')[0])
    cond_dataset = rasterio.open(file)
    cond[layer] = cond_dataset.read(1)



In [52]:
# This function stores the top of the conductor in the dataframe on a click
def onclick_u(event):
    if event.xdata != None and event.ydata != None:
        gdf.at[ind, 'dyke_B_upper'] = event.ydata
        
def onclick_l(event):
    if event.xdata != None and event.ydata != None:
        gdf.at[ind, 'dyke_B_lower'] = event.ydata


def extract_data():
    """
    FUnction for extracting all the AEM data from the netCDF files
    """
    rj_dat = netCDF4.Dataset(row['pmap_path'])

    freq = rj_dat['log10conductivity_histogram'][:].data.astype(np.float)

    cond_pdf = freq / freq.sum(axis =1)[0]

    cond_pdf[cond_pdf == 0] = np.nan
    
    cp_freq = rj_dat["interface_depth_histogram"][:].data.astype(np.float)
    
    cp_pdf = cp_freq / freq.sum(axis =1)[0]
    
    laybins = rj_dat['nlayers_histogram'][:].data
    
    lay_prob = laybins / freq.sum(axis =1)[0]
    
    condmin, condmax = rj_dat.vmin, rj_dat.vmax 
    
    ncond_cells = rj_dat.dimensions['value'].size
    
    cond_cells = np.linspace(condmin, condmax, ncond_cells)
    
    pmin, pmax = rj_dat.pmin, rj_dat.pmax
    
    
    depth_cells = rj_dat['depth'][:].data
    
    extent = [cond_cells.min(), cond_cells.max(), depth_cells.max(), depth_cells.min()]
    
    mean = np.power(10,rj_dat['mean_model'][:].data)
    p10 = np.power(10,rj_dat['p10_model'][:].data)
    p50 = np.power(10,rj_dat['p50_model'][:].data)
    p90 = np.power(10,rj_dat['p90_model'][:].data)
    
    lci_cond = lci_dat['conductivity'][point_ind_lci].data
    lci_depth_top = lci_dat['layer_top_depth'][point_ind_lci].data
    
    lci_doi = lci_dat['depth_of_investigation'][point_ind_lci].data
    
    misfit = np.sqrt(rj_dat['misfit'][:].data/rj_dat.ndata)
    
    burnin = rj_dat.nburnin
    nsamples = rj_dat.nsamples
    sample_no = rj_dat['convergence_sample'][:].data
    nchains = rj_dat.nchains
    
    line = int(rj_dat.line)
    
    return {'conductivity_pdf': cond_pdf, "change_point_pdf": cp_pdf, "conductivity_extent": extent,
           'cond_p10': p10, 'cond_p50': p50, 'cond_p90': p90, 'cond_mean': mean, 'depth_cells': depth_cells,
           'nlayer_bins': laybins, 'nlayer_prob': lay_prob, 'nsamples': nsamples, 'ndata': rj_dat.ndata,
           "nchains": nchains, 'burnin': burnin, 'misfit': misfit, 'sample_no': sample_no, 'cond_cells': cond_cells, 'lci_cond': lci_cond,
           'lci_depth_top': lci_depth_top, 'lci_doi': lci_doi, 'line': line}
    
def HE_plot(D, outfile = None):
    fig = plt.figure(figsize = (12,10))

    # These are for interactive widget mode
    fig.canvas.layout.width = '6in'
    fig.canvas.layout.height= '5in'

    ax1 = fig.add_axes([0.05, 0.35, 0.35, 0.62])
    ax2 = fig.add_axes([0.45, 0.35, 0.2, 0.62])
    ax3 = fig.add_axes([0.70, 0.52, 0.2, 0.2])
    ax4 = fig.add_axes([0.72, 0.32, 0.16, 0.16])
    ax5 = fig.add_axes([0.1, 0.18, 0.76, 0.05])
    ax6 = fig.add_axes([0.1, 0.05, 0.76, 0.13])
    ax7 = fig.add_axes([0.70, 0.78, 0.2, 0.2])
    cbar_ax1 = fig.add_axes([0.05, 0.29, 0.35, 0.01])
    cbar_ax2 = fig.add_axes([0.88, 0.05, 0.01, 0.2])
    
    panel_kwargs = [{'title': '',
                      'color': 'black',
                      'ylabel': 'data \n residual',
                      'legend': False},
                     {'title': 'LCI conductivity',
                      'max_depth': 150.,
                      'shade_doi': True,
                      'colourbar': True,
                      'colourbar_label': 'Conductivity (S/m)',
                      'log_plot': True,
                      'vmin': 0.001,
                      'vmax': .1,
                      'cmap': 'jet',
                      'ylabel': 'elevation \n (mAHD)',
                      'vertical_exaggeration': 5.0}]


    # Plot probability map
    
    im = ax1.imshow(D['conductivity_pdf'], extent = D['conductivity_extent'],
                    aspect = 'auto', cmap = 'rainbow')
    
    #  PLot the median, and percentile plots
    ax1.plot(np.log10(D['cond_p10']), D['depth_cells'], c = 'k',linestyle='dashed', label = 'p10')
    ax1.plot(np.log10(D['cond_p90']), D['depth_cells'], c = 'k',linestyle='dashed', label = 'p90')
    ax1.plot(np.log10(D['cond_p50']), D['depth_cells'], c = 'k',label = 'p50')
    ax1.plot(np.log10(D['cond_mean']), D['depth_cells'], c = 'grey',label = 'mean')
    
    ax1.set_xticklabels([round(10 ** float(x), 4) for x in ax1.get_xticks()])

    # for lci layered model we do some processing
    lci_expanded = np.zeros(shape=2 * len(D['lci_cond']) + 1,
                                 dtype=np.float)

    lci_expanded[1:] = np.repeat(D['lci_cond'], 2)

    depth_expanded = (np.max(D['lci_depth_top']) + 10) * np.ones(shape=len(lci_expanded),
                                                            dtype=np.float)

    depth_expanded[:-1] = np.repeat(D['lci_depth_top'], 2)

    ax1.plot(np.log10(lci_expanded), depth_expanded, c = 'pink',
             linestyle = 'dashed', label = 'lci')
    ax1.plot(ax1.get_xlim(), [D['lci_doi'], D['lci_doi']], c = 'yellow',
             label = 'LCI doi')
    ax1.set_title('rj-MCMC probability map- FID: ' + str(row['fiducial ']))
    ax1.set_ylabel('depth (mBGL)')
    ax1.set_xlabel('Conductivity (S/m)')
    ax1.grid(which = 'both')
    ax1.set_xlim(D['conductivity_extent'][0], D['conductivity_extent'][1] )
    
    ax1.set_ylim(150.,0)#

    
    ax1.legend()
    ax2.plot(D['change_point_pdf'], D['depth_cells'], label = 'P(change point)')
    ax2.set_ylim(ax2.get_ylim()[::-1])
    ax2.set_yticks(np.arange(0, 500, 20.))
    ax2.set_title('change point probability')
    ax2.set_ylim(D['conductivity_extent'][2], D['conductivity_extent'][3])#(100.,0)
    ax2.set_ylim(150.,0)
    ax2.legend()
    ax2.grid(which = 'both')
    
    # Ax3 will be our location
    sample = D['sample_no'][:]
    
    # Add the misfit
    for i in range(D['misfit'].shape[0]):
       
        misfits = D['misfit'][i]
        ax4.plot(sample, misfits)

    ax4.plot([1, D['nsamples']], [1,1], 'k')
    ax4.plot([D['burnin'], D['burnin']],[0.1,1e4], 'k')
    ax4.set_xlim([1, D['nsamples']])
    ax4.set_ylim(0.1, 1e4)

    ax4.set_xscale('log')
    ax4.set_yscale('log')

    ax4.set_xlabel("sample #")
    ax4.set_ylabel("Normalised misfit")
    
    # conductivity plot
    
    ax7.imshow(np.log10(cond[7]), extent = [cond_dataset.bounds[0],
                                  cond_dataset.bounds[2],
                                  cond_dataset.bounds[1], 
                                  cond_dataset.bounds[3]],
              cmap = 'jet',
              vmin = np.log10(panel_kwargs[1]['vmin']*1000.),
              vmax = np.log10(panel_kwargs[1]['vmax']*1000.))
    
    ax7.set_xlim(D['easting'] - 1000., D['easting'] + 1000.)
    ax7.set_ylim(D['northing'] - 1000., D['northing'] + 1000.)
    ax7.plot(D['easting'],D['northing'],  'x', c = 'k')
    
    p1 = [gridded_vars[line]['easting'][0], gridded_vars[line]['easting'][-1]]
    p2 = [gridded_vars[line]['northing'][0], gridded_vars[line]['northing'][-1]]
    ax7.plot(p1, p2, 'k', linewidth = 0.5)
    ax7.set_title('LCI depth slice 61.8-71.6 mBGL', fontsize=10)
    ax7.tick_params(axis='both', which='major', labelsize=8)
    ax7.tick_params(axis='both', which='minor', labelsize=8)
    cb1 = fig.colorbar(im, cax=cbar_ax1, orientation='horizontal')
    cb1.set_label('probabilitiy', fontsize=10)

    res1 = plot_utils.plot_single_line(ax5, gridded_vars[line],
                                       'data_residual', panel_kwargs[0])

    ax5.set_title('LCI conductivity section - ' + str(line))

    im2 = plot_utils.plot_grid(ax6, gridded_vars[line], 'conductivity',
                               panel_kwargs[1])
    
    
    # Add colorbars

    ax6.plot([dist, dist], [-500, 500], 'pink')
    
        
    cb2 = fig.colorbar(im2, cax=cbar_ax2, orientation='vertical')
    
    cb2.ax.set_yticklabels([round(10 ** x, 4) for x in cb2.get_ticks()])
    cb2.set_label('conductivity (S/m)', fontsize=10)
    
    ax5.set_xlim(dist, 1000.,
                 dist + 1000.)
    ax6.set_xlim(dist - 1000., 
                 dist + 1000.)

    
    
    return fig

def gen(df):
    for index, row in df.iterrows():
        yield index, row

In [53]:
#dft = pd.read_csv(r"C:\temp\justin_case.csv")
#gdf = gdf.merge(dft[['fiducial ','dyke_B_upper','dyke_B_lower']], on = 'fiducial ')

,uniqueid,survey,date,flight,line,fiducial,easting,northing,elevation,nchains,nsamples,nburnin,sampletime,misfit_lowest,misfit_average,dyke_B_upper,dyke_B_lower
0,1,1303,20170728,20170729,107501,2415766.0,732671.5,8626858.0,18.4,8,500000,50000,10186.51,2.066913,2.241684,NaN,NaN
1,11,1303,20170728,20170729,107501,2415771.0,732764.3,8626960.0,19.1,8,500000,50000,9814.39,1.654303,1.881077,NaN,NaN
2,21,1303,20170728,20170729,107601,2415933.0,732774.1,8626822.0,19.6,8,500000,50000,8570.84,1.628989,1.761960,65.887097,80.146217
3,31,1303,20170728,20170729,107701,2416683.5,732799.9,8626712.0,18.4,8,500000,50000,8584.71,2.536793,2.655887,53.225806,61.693548
4,41,1303,20170728,20170729,107701,2416688.5,732887.3,8626809.0,20.6,8,500000,50000,8176.06,4.719013,4.962229,39.596772,45.967742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,1805,1303,20170806,20170806,200701,3119267.0,734306.9,8624239.0,26.1,8,500000,50000,6756.93,8.435085,4.940868,NaN,NaN
366,1815,1303,20170806,20170806,200701,3119272.0,734196.9,8624334.0,26.9,8,500000,50000,5612.36,7.739927,4.651870,NaN,NaN
367,1825,1303,20170806,20170806,200701,3119277.0,734084.5,8624417.0,27.2,8,500000,50000,5813.54,6.028025,3.874824,NaN,NaN
368,1835,1303,20170805,20170805,200801,3034156.5,733590.0,8625245.0,26.4,8,500000,50000,5731.28,2.540346,5.866062,NaN,NaN


,uniqueid,survey,date,flight,line,fiducial,easting,northing,elevation,nchains,...,changepoint [145],changepoint [146],changepoint [147],changepoint [148],changepoint [149],pmap_path,dist_along_line,point_ind_lci,dyke_B_upper,dyke_B_lower
88,881,1303,20170726,20170726,113201,2163080.0,735793.9,8621808.0,18.9,8,...,138,101,115,109,108,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...,9536.0,65547,NaN,NaN
89,891,1303,20170726,20170726,113301,2180185.5,735877.2,8621737.0,17.7,8,...,100,110,91,63,68,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...,9520.0,64809,NaN,NaN
90,901,1303,20170726,20170726,113301,2180190.5,735790.7,8621625.0,17.2,8,...,0,1,0,0,0,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...,9376.0,64815,NaN,NaN
91,911,1303,20170726,20170726,113401,2179890.5,735880.6,8621580.0,16.1,8,...,0,1,0,0,0,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...,9520.0,64579,NaN,NaN
92,921,1303,20170726,20170726,113401,2179895.5,735954.2,8621678.0,16.0,8,...,16,18,24,23,25,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...,9648.0,64585,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,1805,1303,20170806,20170806,200701,3119267.0,734306.9,8624239.0,26.1,8,...,3,2,0,2,1,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...,1472.0,1884,NaN,NaN
366,1815,1303,20170806,20170806,200701,3119272.0,734196.9,8624334.0,26.9,8,...,0,2,0,0,1,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...,1328.0,1889,NaN,NaN
367,1825,1303,20170806,20170806,200701,3119277.0,734084.5,8624417.0,27.2,8,...,2,0,2,3,0,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...,1200.0,1894,NaN,NaN
368,1835,1303,20170805,20170805,200801,3034156.5,733590.0,8625245.0,26.4,8,...,0,1,1,2,0,C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\g...,12000.0,5454,NaN,NaN


In [484]:
# Create a generator to iterate through the pandas dataframe

#line = 107501

#n = 1

#df_line = gdf[gdf['line '] == line]

#print(len(df_line))


#print(len(df_line.iloc[::n,:]))

cond_gen = gen(gdf.iloc[88:])#df_line.iloc[::n,:])
#cond_gen = gen(df_line.iloc[-1:,:])


In [488]:
# Define some coordinates to investigate
plt.close('all')

ind, row = next(cond_gen)

point_ind_lci = row['point_ind_lci']

# Get the nearest lci point

easting, northing = row['easting '], row['northing ']

plt.close('all')

D = extract_data()

D['easting'], D['northing'] = easting, northing

line = row['line ']

# Find distance along the lci section
dist = xy_2_var(gridded_vars[line],
                 np.array([[easting, northing]]),
                         'grid_distances')

print(easting)
print(northing)

fig = HE_plot(D)
#plt.savefig(r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\rjplots\\Florina_" + row['BORE_NO'] + ".png", dpi = 150)
cid=  fig.canvas.mpl_connect('button_press_event', onclick_u)

cols = ['uniqueid ', 'survey ', 'date ', 'flight ', 'line ', 'fiducial ',
       'easting ', 'northing ', 'elevation ', 'nchains ', 'nsamples ',
       'nburnin ', 'sampletime ', 'misfit_lowest ', 'misfit_average ',
        'dyke_B_upper', 'dyke_B_lower']

#mask = gdf["topJind_interp"].notnull()

print(ind)

#gdf[cols].to_csv(r"C:\temp\justin_case.csv", index = False)

plt.show()


735880.6
8621580.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

91


C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:127: RuntimeWarning: divide by zero encountered in log10
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:172: RuntimeWarning: invalid value encountered in log10


In [462]:
fig = HE_plot(D)
#plt.savefig(r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\rjplots\\Florina_" + row['BORE_NO'] + ".png", dpi = 150)
cid=  fig.canvas.mpl_connect('button_press_event', onclick_l)

cols = ['uniqueid ', 'survey ', 'date ', 'flight ', 'line ', 'fiducial ',
       'easting ', 'northing ', 'elevation ', 'nchains ', 'nsamples ',
       'nburnin ', 'sampletime ', 'misfit_lowest ', 'misfit_average ',
        'dyke_B_upper', 'dyke_B_lower']


plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:127: RuntimeWarning: divide by zero encountered in log10
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:172: RuntimeWarning: invalid value encountered in log10


In [195]:
index

369

In [42]:
gdf['dyke_B_upper']

0            NaN
1            NaN
2      65.887097
3      53.225806
4      39.596772
         ...    
365          NaN
366          NaN
367          NaN
368          NaN
369          NaN
Name: dyke_B_upper, Length: 370, dtype: float64

In [470]:
np.greater(gdf['dyke_B_lower'].values, gdf['dyke_B_upper'].values)

C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


array([False, False,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
       False,  True, False, False, False, False,  True,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
        True,  True,  True, False,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False,  True, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
        True, False,

In [483]:
mask = gdf['dyke_B_upper'].notnull()

print(mask)

gdf[mask][cols].to_csv(r"C:\Users\PCUser\Desktop\NSC_data\HE_interp\dyke_B_interp.csv", index = False)

0      False
1      False
2       True
3       True
4       True
       ...  
365    False
366     True
367    False
368    False
369    False
Name: dyke_B_upper, Length: 370, dtype: bool


In [474]:
gdf['dyke_B_lower_mAHD'] = gdf['elevation '] - gdf['dyke_B_lower']

In [475]:
gdf['dyke_B_upper_mAHD'] = gdf['elevation '] - gdf['dyke_B_upper']

In [479]:
cols += ['dyke_B_upper', 'dyke_B_lower']


In [482]:
cols = ['uniqueid ',
 'survey ',
 'date ',
 'flight ',
 'line ',
 'fiducial ',
 'easting ',
 'northing ',
 'elevation ',
 'nchains ',
 'nsamples ',
 'nburnin ',
 'sampletime ',
 'misfit_lowest ',
 'misfit_average ',
 'dyke_B_upper',
 'dyke_B_lower',
 'dyke_B_upper_mAHD',
 'dyke_B_lower_mAHD']